In [ ]:
# Import libraries
import pandas as pd
import numpy as np

# Load the datasets
social_profiles = pd.read_csv('../Dataset/customer_social_profiles - customer_social_profiles.csv')
transactions = pd.read_csv('../Dataset/customer_transactions - customer_transactions.csv')

print("=== Social Profiles Sample ===")
print(social_profiles.head())
print("\n=== Transactions Sample ===")
print(transactions.head())


=== Social Profiles Sample ===
  customer_id_new social_media_platform  engagement_score  \
0            A178              LinkedIn                74   
1            A190               Twitter                82   
2            A150              Facebook                96   
3            A162               Twitter                89   
4            A197               Twitter                92   

   purchase_interest_score review_sentiment  
0                      4.9         Positive  
1                      4.8          Neutral  
2                      1.6         Positive  
3                      2.6         Positive  
4                      2.3          Neutral  

=== Transactions Sample ===
   customer_id_legacy  transaction_id  purchase_amount purchase_date  \
0                 151            1001              408    2024-01-01   
1                 192            1002              332    2024-01-02   
2                 114            1003              442    2024-01-03   
3        

In [2]:
# Check for nulls and duplicates BEFORE processing
print("=== Social Profiles - Nulls ===")
print(social_profiles.isnull().sum())
print("\n=== Social Profiles - Duplicates ===")
print(f"Total duplicate rows: {social_profiles.duplicated().sum()}")

print("\n=== Transactions - Nulls ===")
print(transactions.isnull().sum())
print("\n=== Transactions - Duplicates ===")
print(f"Total duplicate rows: {transactions.duplicated().sum()}")


=== Social Profiles - Nulls ===
customer_id_new            0
social_media_platform      0
engagement_score           0
purchase_interest_score    0
review_sentiment           0
dtype: int64

=== Social Profiles - Duplicates ===
Total duplicate rows: 5

=== Transactions - Nulls ===
customer_id_legacy     0
transaction_id         0
purchase_amount        0
purchase_date          0
product_category       0
customer_rating       10
dtype: int64

=== Transactions - Duplicates ===
Total duplicate rows: 0


In [ ]:
# Handle duplicates in social_profiles

social_profiles = social_profiles.drop_duplicates()
print(f"Social profiles after removing duplicates: {social_profiles.shape[0]} rows")

# Handle duplicates in transactions

transactions = transactions.drop_duplicates()
print(f"Transactions after removing duplicates: {transactions.shape[0]} rows")


Social profiles after removing duplicates: 150 rows
Transactions after removing duplicates: 150 rows


In [ ]:
# Fix data types
# Standardize customer_id_new.
# Match customer_id_new with customer_id_legacy
social_profiles['customer_id'] = social_profiles['customer_id_new'].str.replace('A', '').astype(int)

# Transactions: Rename customer_id_legacy to customer_id for merge
transactions['customer_id'] = transactions['customer_id_legacy'].astype(int)

# Fix date type in transactions
transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])

# Fix numeric types
social_profiles['engagement_score'] = pd.to_numeric(social_profiles['engagement_score'], errors='coerce')
social_profiles['purchase_interest_score'] = pd.to_numeric(social_profiles['purchase_interest_score'], errors='coerce')
transactions['purchase_amount'] = pd.to_numeric(transactions['purchase_amount'], errors='coerce')
transactions['customer_rating'] = pd.to_numeric(transactions['customer_rating'], errors='coerce')

print("Data types fixed successfully!")
print("\nSocial Profiles dtypes:")
print(social_profiles.dtypes)
print("\nTransactions dtypes:")
print(transactions.dtypes)


Data types fixed successfully!

Social Profiles dtypes:
customer_id_new             object
social_media_platform       object
engagement_score             int64
purchase_interest_score    float64
review_sentiment            object
customer_id                  int64
dtype: object

Transactions dtypes:
customer_id_legacy             int64
transaction_id                 int64
purchase_amount                int64
purchase_date         datetime64[ns]
product_category              object
customer_rating              float64
customer_id                    int64
dtype: object


In [ ]:
# Handle nulls in customer_rating (transactions)

null_count_before = transactions['customer_rating'].isnull().sum()
median_rating = transactions['customer_rating'].median()
transactions['customer_rating'] = transactions['customer_rating'].fillna(median_rating)
print(f"Filled {null_count_before} null values in customer_rating with median: {median_rating:.2f}")

# Check for any remaining nulls
print("\n=== Remaining Nulls After Processing ===")
print("Social Profiles:")
print(social_profiles.isnull().sum())
print("\nTransactions:")
print(transactions.isnull().sum())


Filled 10 null values in customer_rating with median: 3.00

=== Remaining Nulls After Processing ===
Social Profiles:
customer_id_new            0
social_media_platform      0
engagement_score           0
purchase_interest_score    0
review_sentiment           0
customer_id                0
dtype: int64

Transactions:
customer_id_legacy    0
transaction_id        0
purchase_amount       0
purchase_date         0
product_category      0
customer_rating       0
customer_id           0
dtype: int64


In [ ]:
# JOIN LOGIC JUSTIFICATION:
# Use INNER JOIN on customer_id to keep customers with both social profiles and transactions, ensuring complete data for model training.

# Perform the merge
merged_data = pd.merge(
    social_profiles,
    transactions,
    on='customer_id',
    how='inner'  # Inner join to keep only customers present in both datasets
)

print(f"Merged dataset shape: {merged_data.shape}")
print(f"\nMerge justification: Using INNER JOIN to combine social profiles with transaction history.")
print(f"This creates {merged_data.shape[0]} rows representing customer-platform-transaction combinations.")


Merged dataset shape: (213, 12)

Merge justification: Using INNER JOIN to combine social profiles with transaction history.
This creates 213 rows representing customer-platform-transaction combinations.


In [9]:
# CHECKS

# 1. Check for nulls after merge
print("1. Null values check:")
print(merged_data.isnull().sum())
print(f"\nTotal nulls: {merged_data.isnull().sum().sum()}")

# 2. Check for duplicates
print("\n2. Duplicate rows check:")
print(f"Total duplicate rows: {merged_data.duplicated().sum()}")

# 3. Check data types
print("\n3. Data types:")
print(merged_data.dtypes)

# 4. Check unique customers
print("\n4. Unique customers in merged data:")
print(f"Unique customers: {merged_data['customer_id'].nunique()}")
print(f"Unique social profile entries: {social_profiles['customer_id'].nunique()}")
print(f"Unique transaction customers: {transactions['customer_id'].nunique()}")

# 5. Check feature ranges
print("\n5. Feature ranges:")
print(f"Engagement Score: {merged_data['engagement_score'].min():.1f} - {merged_data['engagement_score'].max():.1f}")
print(f"Purchase Interest Score: {merged_data['purchase_interest_score'].min():.1f} - {merged_data['purchase_interest_score'].max():.1f}")
print(f"Purchase Amount: ${merged_data['purchase_amount'].min():.2f} - ${merged_data['purchase_amount'].max():.2f}")
print(f"Customer Rating: {merged_data['customer_rating'].min():.1f} - {merged_data['customer_rating'].max():.1f}")

# 6. Display sample of merged data
print("\n6. Sample of merged data:")
print(merged_data.head(10))


1. Null values check:
customer_id_new            0
social_media_platform      0
engagement_score           0
purchase_interest_score    0
review_sentiment           0
customer_id                0
customer_id_legacy         0
transaction_id             0
purchase_amount            0
purchase_date              0
product_category           0
customer_rating            0
dtype: int64

Total nulls: 0

2. Duplicate rows check:
Total duplicate rows: 0

3. Data types:
customer_id_new                    object
social_media_platform              object
engagement_score                    int64
purchase_interest_score           float64
review_sentiment                   object
customer_id                         int64
customer_id_legacy                  int64
transaction_id                      int64
purchase_amount                     int64
purchase_date              datetime64[ns]
product_category                   object
customer_rating                   float64
dtype: object

4. Unique custom

In [8]:
# Final dataset summary
print("=== FINAL MERGED DATASET SUMMARY ===\n")
print(f"Shape: {merged_data.shape[0]} rows, {merged_data.shape[1]} columns")
print(f"\nColumns: {merged_data.columns.tolist()}")
print(f"\nTarget variable for prediction: 'product_category'")
print(f"Unique product categories: {merged_data['product_category'].unique()}")
print(f"\nThe merged dataset is ready for predictive modeling!")


=== FINAL MERGED DATASET SUMMARY ===

Shape: 213 rows, 12 columns

Columns: ['customer_id_new', 'social_media_platform', 'engagement_score', 'purchase_interest_score', 'review_sentiment', 'customer_id', 'customer_id_legacy', 'transaction_id', 'purchase_amount', 'purchase_date', 'product_category', 'customer_rating']

Target variable for prediction: 'product_category'
Unique product categories: ['Groceries' 'Sports' 'Books' 'Electronics' 'Clothing']

The merged dataset is ready for predictive modeling!


In [10]:

 merged_data.to_csv('../Dataset/merged_customer_data.csv', index=False)
 print("Merged dataset saved successfully!")


Merged dataset saved successfully!
